### Fine Tuning

In [ ]:
import finetuner
from docarray import DocumentArray, Document

In [ ]:
# login with same credentials which you used to push private dataset to jina cloud
# for public dataset just use any login
finetuner.notebook_login(force=True)

### Finetune Now

In [ ]:
# finetuner.notebook_login()
run_name = 'unsplash-lite-clip-run-onnx-11132022-2145'

In [ ]:
run = finetuner.get_run(run.name)

In [ ]:
run = finetuner.fit(
    model='openai/clip-vit-base-patch32',
    train_data='unsplash-lite-clean-train-data-clip',
    eval_data='unsplash-lite-clean-eval-data-clip',
    run_name=run_name,
    epochs=5,
    learning_rate= 1e-7,
    loss='CLIPLoss',
    device='cuda',
    to_onnx=True,#MAKE SURE TO CHOOSE FOR ONNX
)

In [ ]:
# note, the fine-tuning might takes many hours
for entry in run.stream_logs():
    print(entry)

In [ ]:
run = finetuner.get_run(run.name)

In [ ]:
# note, the fine-tuning might takes many hours
for entry in run.stream_logs():
    print(entry)

In [ ]:
artifact_name = 'unsplash-lite-clean-clip-onnx-model'
artifact = run.save_artifact(artifact_name)

##### Quick Check Downloaded aritifact for Inference with ONNX

In [ ]:
artifact_local_path = f"{artifact_name}/{run_name}.zip"

In [ ]:
text_da = DocumentArray([Document(text='some text to encode')])
image_da = DocumentArray([Document(uri='https://upload.wikimedia.org/wikipedia/commons/4/4e/Single_apple.png')])

clip_text_encoder = finetuner.get_model(artifact=artifact_local_path, select_model='clip-text', is_onnx=True)
clip_image_encoder = finetuner.get_model(artifact=artifact_local_path, select_model='clip-vision', is_onnx=True)

finetuner.encode(model=clip_text_encoder, data=text_da)
finetuner.encode(model=clip_image_encoder, data=image_da)

print(text_da.embeddings.shape)
print(image_da.embeddings.shape)

In [ ]:
print(text_da.summary())

In [ ]:
print(image_da.summary())

##### For Backend Server to run your FineTuned Onnx model on Clip Server
* Make sure finetuned_clip.yml exist in backend directory
* create a new directory for unsplash-finetuned-onnx
* unzip the unsplash-lite-clip-run-onnx-11132022-2145.zip
* go to models/clip-text and copy models.onnx as textual.onnx to new unsplash-finetuned-onnx directory
* go to models/clip-vision and copy models.onnx as visual.onnx to new unsplash-finetuned-onnx directory